# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'05-09-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'05-09-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-05-10 04:20:38,33.93911,67.709953,61842,2686,54222.0,4934.0,Afghanistan,158.861124,4.343327
1,NaN,NaN,NaN,Albania,2021-05-10 04:20:38,41.15330,20.168300,131723,2412,117089.0,12222.0,Albania,4577.211759,1.831115
2,NaN,NaN,NaN,Algeria,2021-05-10 04:20:38,28.03390,1.659600,124104,3328,86420.0,34356.0,Algeria,283.012653,2.681622
3,NaN,NaN,NaN,Andorra,2021-05-10 04:20:38,42.50630,1.521800,13423,127,12995.0,301.0,Andorra,17372.678444,0.946137
4,NaN,NaN,NaN,Angola,2021-05-10 04:20:38,-11.20270,17.873900,28740,633,24717.0,3390.0,Angola,87.445280,2.202505


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,59745,59939,60122,60300,60563,60797,61162,61455,61755,61842
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,131085,131185,131238,131276,131327,131419,131510,131577,131666,131723
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,122108,122311,122522,122717,122999,123272,123473,123692,123900,124104


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2625,2631,2637,2642,2648,2654,2664,2673,2683,2686
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2394,2396,2397,2399,2402,2403,2406,2408,2411,2412
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3253,3261,3270,3280,3289,3299,3307,3315,3321,3328


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,53206,53272,53391,53391,53694,53750,53961,54019,54040,54222
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,109338,110172,110920,111792,112626,113468,114362,115253,116126,117089
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,85108,85249,85398,85534,85693,85869,86007,86149,86280,86420


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6904,6907,6909,6910,6910,6914,6914,6918,6918,6920
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20941,20966,20983,20993,20993,21035,21093,21107,21123,21131
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2300,2302,2302,2302,2302,2307,2307,2307,2307,2308


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,107,107,107,107,107,108,108,108,108,108
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,305,306,306,306,306,307,307,307,308,308
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,56,56,56,56,56,56,57,57,57,57


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-05-10 04:20:38,32.539527,-86.644082,6920,108,NaN,NaN,"Autauga, Alabama, US",12386.117525,1.560694
689,1075.0,Lamar,Alabama,US,2021-05-10 04:20:38,33.779950,-88.096680,1425,35,NaN,NaN,"Lamar, Alabama, US",10322.346976,2.456140
690,1077.0,Lauderdale,Alabama,US,2021-05-10 04:20:38,34.901719,-87.656247,9424,241,NaN,NaN,"Lauderdale, Alabama, US",10162.947945,2.557301


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,32707641,581754,0.0
India,22296414,242362,18317404.0
Brazil,15184790,422340,13457400.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,32707641,581754,0.0,32125887.0,2021-05-10 04:20:38,37.950547,-91.419547
India,22296414,242362,18317404.0,3736648.0,2021-05-10 04:20:38,23.088275,81.806127
Brazil,15184790,422340,13457400.0,1305050.0,2021-05-10 04:20:38,-12.669522,-48.480493
France,5838294,106553,371478.0,5360263.0,2021-05-10 04:20:38,6.430808,-34.730285
Turkey,5031332,43029,4716918.0,271385.0,2021-05-10 04:20:38,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3758028,62314,0.0
Texas,2914504,50643,0.0
Florida,2269806,35731,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3758028,62314,0.0,3695714.0,2021-05-10 04:20:38,37.843962,-120.728594
Texas,2914504,50643,0.0,2863861.0,2021-05-10 04:20:38,31.660643,-98.653069
Florida,2269806,35731,0.0,2234075.0,2021-05-10 04:20:38,28.940755,-82.700744
New York,2072426,52688,0.0,2019738.0,2021-05-10 04:20:38,42.544151,-75.474183
Illinois,1354869,24577,0.0,1330292.0,2021-05-10 04:20:38,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1235651,24022,0.0
Illinois,Cook,542450,10056,0.0
Arizona,Maricopa,541555,9922,0.0
Florida,Miami-Dade,490499,6252,0.0
Texas,Harris,395304,6275,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1235651,24022,0.0,1211629.0,2021-05-10 04:20:38,34.308284,-118.228241,6037.0
Illinois,Cook,542450,10056,0.0,532394.0,2021-05-10 04:20:38,41.841448,-87.816588,17031.0
Arizona,Maricopa,541555,9922,0.0,531633.0,2021-05-10 04:20:38,33.348359,-112.491815,4013.0
Florida,Miami-Dade,490499,6252,0.0,484247.0,2021-05-10 04:20:38,25.611236,-80.551706,12086.0
Texas,Harris,395304,6275,0.0,389029.0,2021-05-10 04:20:38,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,61455,131577,123692,13390,28201,1232,3118134,218681,29906,628817,...,4446752,216146,93597,4,205181,3137,300946,6446,92004,38403
2021-05-08,61755,131666,123900,13406,28477,1232,3136158,219092,29922,630050,...,4448808,218800,93978,4,206549,3245,301437,6462,92057,38414
2021-05-09,61842,131723,124104,13423,28740,1231,3147740,219270,29931,631076,...,4450578,220683,94397,4,207870,3412,301751,6482,92092,38419


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,2673,2408,3315,127,628,32,66872,4209,910,10351,...,127858,3032,660,1,2263,35,3338,1269,1256,1576
2021-05-08,2683,2411,3321,127,630,32,67042,4225,910,10374,...,127863,3082,662,1,2280,35,3351,1270,1257,1576
2021-05-09,2686,2412,3328,127,633,32,67325,4234,910,10382,...,127865,3122,662,1,2291,35,3358,1271,1257,1576


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,54019,115253,86149,12936,24698,1014,2777902,203307,23407,599854,...,14961,187153,89338,3,187665,2560,283570,2978,90316,36041
2021-05-08,54040,116126,86280,12962,24713,1014,2798328,204121,23429,601958,...,15011,189739,89703,3,188947,2602,285575,2983,90363,36052
2021-05-09,54222,117089,86420,12995,24717,1168,2817165,204578,23437,603901,...,15011,192114,90053,3,190768,2602,286385,2992,90428,36063


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6904,6907,6909,6910,6910,6914,6914,6918,6918,6920
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1408,1409,1411,1412,1412,1415,1418,1419,1425,1425
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9376,9386,9393,9397,9397,9407,9413,9419,9420,9424
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2992,2994,2996,2996,2996,2997,2998,2998,3002,3002
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15784,15792,15796,15800,15800,15822,15844,15856,15873,15884


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-07,530325,68880,0,867443,337170,3755869,520816,342718,105775,49,...,852841,2911927,399760,23393,3152,665332,412483,155680,665210,58623
2021-05-08,530744,68880,0,868382,337415,3757115,522382,342718,105948,49,...,852841,2913480,400120,23445,3210,666111,413980,156001,665842,58623
2021-05-09,530988,68880,0,868830,337510,3758028,523309,342718,106148,49,...,852841,2914504,400352,23486,3210,666650,413980,156406,666180,58623


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-05-07        6918   21107    2307  2604   6651    1230   2159   14488   
2021-05-08        6918   21123    2307  2605   6656    1230   2159   14496   
2021-05-09        6920   21131    2308  2607   6660    1230   2159   14501   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-07         3559     1844  ...    2728    624     3157      782   
2021-05-08         3560     1846  ...    2728    624     3157      782   
2021-05-09         3563     1848  ...    2728    624     3157      782   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-07           4285  3751  2217          0      911    651  
2021-05-08           4285  3751  2217          0      911    651  
2021-05-09           4285  3751  2217          0      911    651  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-07,10966,349,0,17391,5759,62220,6352,8137,1629,0,...,12255,50583,2223,249,27,10874,5564,2719,7625,710
2021-05-08,10978,349,0,17407,5760,62280,6355,8137,1636,0,...,12255,50626,2223,249,27,10885,5564,2726,7652,710
2021-05-09,10978,349,0,17409,5761,62314,6357,8137,1636,0,...,12255,50643,2224,249,27,10895,5564,2726,7652,710


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-05-07         108     307      57   64    137      41     69     314   
2021-05-08         108     308      57   64    137      41     69     314   
2021-05-09         108     308      57   64    137      41     69     314   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-05-07          123       45  ...      30     11       31        7   
2021-05-08          123       45  ...      30     11       31        7   
2021-05-09          123       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-07             38     9    13          0       26      5  
2021-05-08             38     9    13          0       26      5  
2021-05-09             38     9    13          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,0.004791,0.000509,0.001774,0.002021,0.010028,0.000000,0.007285,0.001631,0.000435,0.002124,...,0.000560,0.012635,0.004518,0.0,0.005508,0.015210,0.001861,0.003112,0.000631,0.000130
2021-05-08,0.004882,0.000676,0.001682,0.001195,0.009787,0.000000,0.005780,0.001879,0.000535,0.001961,...,0.000462,0.012279,0.004071,0.0,0.006667,0.034428,0.001632,0.002482,0.000576,0.000286
2021-05-09,0.001409,0.000433,0.001646,0.001268,0.009236,-0.000812,0.003693,0.000812,0.000301,0.001628,...,0.000398,0.008606,0.004458,0.0,0.006396,0.051464,0.001042,0.003095,0.000380,0.000130


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,0.003378,0.000831,0.002419,0.0,0.009646,0.0,0.009191,0.004055,0.0,0.001742,...,0.000117,0.020188,0.004566,0.0,0.008467,0.0,0.003608,0.003162,0.000000,0.000635
2021-05-08,0.003741,0.001246,0.001810,0.0,0.003185,0.0,0.002542,0.003801,0.0,0.002222,...,0.000039,0.016491,0.003030,0.0,0.007512,0.0,0.003895,0.000788,0.000796,0.000000
2021-05-09,0.001118,0.000415,0.002108,0.0,0.004762,0.0,0.004221,0.002130,0.0,0.000771,...,0.000016,0.012979,0.000000,0.0,0.004825,0.0,0.002089,0.000787,0.000000,0.000000


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,0.001075,0.007791,0.001651,0.002791,0.007958,0.000000,0.007497,0.005112,0.000470,0.003231,...,0.000000,0.014346,0.004080,0.0,0.006851,0.000000,0.007046,0.002356,0.000576,0.000389
2021-05-08,0.000389,0.007575,0.001521,0.002010,0.000607,0.000000,0.007353,0.004004,0.000940,0.003508,...,0.003342,0.013818,0.004086,0.0,0.006831,0.016406,0.007071,0.001679,0.000520,0.000305
2021-05-09,0.003368,0.008293,0.001623,0.002546,0.000162,0.151874,0.006732,0.002239,0.000341,0.003228,...,0.000000,0.012517,0.003902,0.0,0.009638,0.000000,0.002836,0.003017,0.000719,0.000305


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-07,0.000592,0.001716,NaN,0.000946,0.000659,0.000718,0.002699,0.001274,0.001923,0.0,...,0.000903,0.000980,0.000967,0.004164,0.000000,0.001412,0.003425,0.002182,0.001127,0.00164
2021-05-08,0.000790,0.000000,NaN,0.001082,0.000727,0.000332,0.003007,0.000000,0.001636,0.0,...,0.000000,0.000533,0.000901,0.002223,0.018401,0.001171,0.003629,0.002062,0.000950,0.00000
2021-05-09,0.000460,0.000000,NaN,0.000516,0.000282,0.000243,0.001775,0.000000,0.001888,0.0,...,0.000000,0.000351,0.000580,0.001749,0.000000,0.000809,0.000000,0.002596,0.000508,0.00000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-05-07      0.000579  0.000664  0.000000  0.000000  0.000903     0.0   
2021-05-08      0.000000  0.000758  0.000000  0.000384  0.000752     0.0   
2021-05-09      0.000289  0.000379  0.000433  0.000768  0.000601     0.0   

Province_State                                          ...   Wyoming         \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park Platte   
2021-05-07      0.000463  0.000483  0.000000  0.000543  ...  0.003679    0.0   
2021-05-08      0.000000  0.000552  0.000281  0.001085  ...  0.000000    0.0   
2021-05-09      0.000000  0.000345  0.000843  0.001083  ...  0.000000    0.0   

Province_State                                                               \
Admin2          Sheridan Sublette Sweetwater     Teton     Uinta Unassigned   
2021-05-07      0.001586      0.0   0.002105 -0.000533  0.002714        NaN   
2021-05-08      0.000000      0.0   0.000000  0.000000  0.000000        NaN   
2021-05-09      0.000000      0.0   0.000000  0.000000  0.000000        NaN   

Province_State                  
Admin2         Washakie Weston  
2021-05-07          0.0    0.0  
2021-05-08          0.0    0.0  
2021-05-09          0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-07,0.001827,0.0,NaN,0.001382,0.000347,0.001062,0.001103,0.000738,0.000000,NaN,...,0.000817,0.001108,0.001803,0.0,0.0,0.001197,0.001981,0.002212,0.001445,0.0
2021-05-08,0.001094,0.0,NaN,0.000920,0.000174,0.000964,0.000472,0.000000,0.004297,NaN,...,0.000000,0.000850,0.000000,0.0,0.0,0.001012,0.000000,0.002574,0.003541,0.0
2021-05-09,0.000000,0.0,NaN,0.000115,0.000174,0.000546,0.000315,0.000000,0.000000,NaN,...,0.000000,0.000336,0.000450,0.0,0.0,0.000919,0.000000,0.000000,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                       \
Admin2         Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2021-05-07         0.0  0.000000     0.0  0.015873  0.007353     0.0    0.0   
2021-05-08         0.0  0.003257     0.0  0.000000  0.000000     0.0    0.0   
2021-05-09         0.0  0.000000     0.0  0.000000  0.000000     0.0    0.0   

Province_State                              ... Wyoming                  \
Admin2           Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-05-07      0.003195      0.0      0.0  ...     0.0    0.0      0.0   
2021-05-08      0.000000      0.0      0.0  ...     0.0    0.0      0.0   
2021-05-09      0.000000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-05-07          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-08          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-05-09          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,0.004841,0.000568,0.001821,0.001877,0.010426,0.000002,0.007464,0.001782,0.000432,0.002171,...,0.000541,0.013797,0.004501,2.543132e-06,0.005658,0.014672,0.002112,0.002953,0.000722,0.000511
2021-05-08,0.004861,0.000622,0.001751,0.001536,0.010106,0.000001,0.006622,0.001831,0.000484,0.002066,...,0.000502,0.013038,0.004286,1.271566e-06,0.006162,0.024550,0.001872,0.002718,0.000649,0.000399
2021-05-09,0.003135,0.000528,0.001699,0.001402,0.009671,-0.000405,0.005158,0.001322,0.000392,0.001847,...,0.000450,0.010822,0.004372,6.357829e-07,0.006279,0.038007,0.001457,0.002906,0.000515,0.000264


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,0.003192,0.000910,0.002547,0.000258,0.009103,0.000008,0.008153,0.003687,1.639292e-11,0.001941,...,0.000117,0.020146,0.003363,0.0,0.008322,1.758530e-76,0.003786,0.004898,0.000379,0.000605
2021-05-08,0.003467,0.001078,0.002179,0.000129,0.006144,0.000004,0.005348,0.003744,8.196458e-12,0.002082,...,0.000078,0.018318,0.003197,0.0,0.007917,8.792649e-77,0.003840,0.002843,0.000588,0.000303
2021-05-09,0.002292,0.000746,0.002143,0.000065,0.005453,0.000002,0.004784,0.002937,4.098229e-12,0.001426,...,0.000047,0.015648,0.001598,0.0,0.006371,4.396325e-77,0.002965,0.001815,0.000294,0.000151


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-05-07,0.002067,0.007730,0.001706,0.002680,0.006086,0.000006,0.007577,0.004156,0.000665,0.003395,...,0.002242,0.014603,0.004108,0.000061,0.006660,0.000242,0.006797,0.004216,0.000583,0.002084
2021-05-08,0.001228,0.007652,0.001613,0.002345,0.003347,0.000003,0.007465,0.004080,0.000802,0.003451,...,0.002792,0.014210,0.004097,0.000031,0.006746,0.008324,0.006934,0.002948,0.000552,0.001195
2021-05-09,0.002298,0.007972,0.001618,0.002445,0.001754,0.075938,0.007098,0.003159,0.000572,0.003340,...,0.001396,0.013364,0.003999,0.000015,0.008192,0.004162,0.004885,0.002982,0.000636,0.000750


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-07,0.000751,0.001708,NaN,0.000852,0.000657,0.000633,0.002824,0.001594,0.001810,1.005805e-126,...,0.001057,0.000949,0.000967,0.003169,0.000387,0.001332,0.003284,0.002320,0.001135,0.001525
2021-05-08,0.000770,0.000854,NaN,0.000967,0.000692,0.000482,0.002915,0.000797,0.001723,5.029027e-127,...,0.000528,0.000741,0.000934,0.002696,0.009394,0.001252,0.003456,0.002191,0.001042,0.000763
2021-05-09,0.000615,0.000427,NaN,0.000741,0.000487,0.000363,0.002345,0.000398,0.001805,2.514513e-127,...,0.000264,0.000546,0.000757,0.002222,0.004697,0.001030,0.001728,0.002393,0.000775,0.000381


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-07      0.000381  0.001316  0.000282  0.000360  0.001123  0.000209   
2021-05-08      0.000191  0.001037  0.000141  0.000372  0.000937  0.000104   
2021-05-09      0.000240  0.000708  0.000287  0.000570  0.000769  0.000052   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-05-07      0.000476  0.000580  0.000649  0.000772  ...  0.002383   
2021-05-08      0.000238  0.000566  0.000465  0.000928  ...  0.001191   
2021-05-09      0.000119  0.000456  0.000654  0.001006  ...  0.000596   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-05-07      0.000407  0.001725  0.000931    0.00228  0.000531  0.002305   
2021-05-08      0.000204  0.000862  0.000465    0.00114  0.000265  0.001153   
2021-05-09      0.000102  0.000431  0.000233    0.00057  0.000133  0.000576   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-05-07           -1.0  0.000797  0.000053  
2021-05-08           -1.0  0.000399  0.000026  
2021-05-09           -1.0  0.000199  0.000013  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-05-07,0.001494,0.000818,NaN,0.000887,0.000438,0.001070,0.001369,0.000798,0.000321,NaN,...,0.000957,0.001128,0.001411,0.001173,7.335957e-08,0.001410,0.002172,0.002452,0.001698,0.000267
2021-05-08,0.001294,0.000409,NaN,0.000903,0.000306,0.001017,0.000921,0.000399,0.002309,NaN,...,0.000479,0.000989,0.000706,0.000586,3.667978e-08,0.001211,0.001086,0.002513,0.002620,0.000133
2021-05-09,0.000647,0.000204,NaN,0.000509,0.000240,0.000781,0.000618,0.000200,0.001154,NaN,...,0.000239,0.000662,0.000578,0.000293,1.833989e-08,0.001065,0.000543,0.001257,0.001310,0.000067


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-05-07      0.001168  0.000435  0.004465  0.007953  0.004617  0.000195   
2021-05-08      0.000584  0.001846  0.002232  0.003976  0.002309  0.000098   
2021-05-09      0.000292  0.000923  0.001116  0.001988  0.001154  0.000049   

Province_State                                                  ...  \
Admin2                Butler   Calhoun  Chambers      Cherokee  ...   
2021-05-07      9.055959e-07  0.001998  0.000096  8.678430e-08  ...   
2021-05-08      4.527980e-07  0.000999  0.000048  4.339215e-08  ...   
2021-05-09      2.263990e-07  0.000499  0.000024  2.169608e-08  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-05-07      4.900940e-16  1.989273e-32  3.701496e-18  1.626101e-38   
2021-05-08      2.450470e-16  9.946364e-33  1.850748e-18  8.130503e-39   
2021-05-09      1.225235e-16  4.973182e-33  9.253741e-19  4.065252e-39   

Province_State                                                              \
Admin2         Sweetwater         Teton     Uinta Unassigned      Washakie   
2021-05-07       0.001689  5.190819e-26  0.005208  -0.501961  1.010324e-30   
2021-05-08       0.000845  2.595409e-26  0.002604  -0.501961  5.051618e-31   
2021-05-09       0.000422  1.297705e-26  0.001302  -0.501961  2.525809e-31   

Province_State                
Admin2                Weston  
2021-05-07      2.145702e-29  
2021-05-08      1.072851e-29  
2021-05-09      5.364254e-30  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
